In [1]:
import pandas as pd
import os

In [ ]:
file_names = [ '2021_incidents.csv', '2022_incidents.csv', '2023_incidents.csv']

dataframes_list = []

print("Starting merge process...")

for file in file_names:
    if os.path.exists(file):
        try:
            # Read the CSV file
            df = pd.read_csv(file)
            # Add it to our list
            dataframes_list.append(df)
            print(f"Successfully loaded: {file} ({len(df)} rows)")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    else:
        print(f"Warning: File not found - {file}")

# Check if we have data to merge
if dataframes_list:
    # Concatenate all dataframes in the list
    # ignore_index=True resets the index so it goes from 0 to N continuously
    merged_df = pd.concat(dataframes_list, ignore_index=True)

    # Save to a new CSV file
    output_filename = 'merged_incidents.csv'
    merged_df.to_csv(output_filename, index=False)
    
    print(f"\nSuccess! Merged {len(dataframes_list)} files.")
    print(f"Total rows: {len(merged_df)}")
    print(f"Saved as: {output_filename}")
else:
    print("No files were loaded. Nothing to merge.")
